# Feature extraction in MASS
we are starting from the results of the 29 iterations of wrapper feature selection in DREAMS

In [1]:
#external libraries
import os
import dotenv
import pandas as pd
import numpy as np
from scipy import signal as sg
import pickle as pkl
import time
import matplotlib.pyplot as plt

#project library
from spinco import *

#environment variables
dotenv.load_dotenv('lab.env')

#project variables
datapath=os.environ['DATAPATH']

In [2]:
#define features path
masspath=datapath+"\MASS"
featurespath=masspath+"\\features"
        
windowDurations=[0.5,1,1.5,2]

## Load data

In [3]:
#load data THIS NEEDS REFINEMENT AND CONVERGENCE TO USE WITH MULTIPLE DATABASES
def loadMASSSpindles(path):
    #signalsMetadata
    signalsMetadata=pd.read_csv(path+'\\signals\\signalsMetadata.csv')
    signalsMetadata['subjectId']=signalsMetadata.apply(
        lambda row: str(row.subjectId).zfill(4),axis=1)

    #load signals from pickle
    signals={}
    for index, row in signalsMetadata.iterrows():
        signalpath=path+"/signals/"+row.file
        cFile = open(signalpath, 'rb')
        signals[row.subjectId]= pkl.load(cFile)
        cFile.close()

    #spindle annotations
    annotations=pd.read_csv(path+'\\annotations\\annotations.csv')
    annotations['subjectId']=annotations.apply(
        lambda row: str(row.subjectId).zfill(4),axis=1)
    annotations['labelerId']=annotations.apply(
        lambda row: str(row.labelerId).zfill(4),axis=1)
    
    #add stop and index colums
    annotations=annotations.merge(signalsMetadata[['subjectId','samplerate']],how='left',on='subjectId')
    annotations['stopTime']=annotations.apply(
        lambda row: row.startTime+row.duration , axis=1)
    annotations['startInd']=annotations.apply(
        lambda row: seconds2index(row.startTime,row.samplerate) , axis=1)
    annotations['stopInd']=annotations.apply(
        lambda row: seconds2index(row.stopTime,row.samplerate) , axis=1)

    
    return signals, annotations, signalsMetadata
    

In [4]:
signals, annotations, signalsMetadata = loadMASSSpindles(masspath)

In [5]:
signalsMetadata.head(5)

,subjectId,file,channel,duration,samplerate
0,0001,MASS_0001.pkl,C3-CLE,28956.0,256
1,0002,MASS_0002.pkl,C3-CLE,35016.0,256
2,0003,MASS_0003.pkl,C3-CLE,36760.0,256
3,0004,MASS_0004.pkl,C3-CLE,28004.0,256
4,0005,MASS_0005.pkl,C3-CLE,31244.0,256


In [6]:
samplerate=256

## Preprocess

In [7]:
def preprocessVector(vector,samplerate):
    #1. Lowpass 50Hz
    vector=filterBand(vector,[0,50],samplerate,filterOrder=4)
    #2. Z-score
    m=np.mean(vector)
    s=np.std(vector)
    vector=(vector-m)/s
    return vector

In [8]:
for subject,signal in signals.items():
    signals[subject]=preprocessVector(signal,samplerate)

## Extract features

### folder estructure

In [9]:
fspath=featurespath+'/'+str(samplerate)+'fs/'
if not os.path.isdir(fspath):
    os.mkdir(fspath)
for window in windowDurations:
    windowPath=fspath+str(window)+'win'
    if not os.path.isdir(windowPath):
        os.mkdir(windowPath)
    for subject in signalsMetadata.subjectId:
        subjectPath=windowPath+'/'+subject
        if not os.path.isdir(subjectPath):
            os.mkdir(subjectPath)

### band definition

In [10]:
bands={
    'delta1':[0.1,2],
    'delta2':[2,4],
    'theta':[4,8],
    'alpha':[8,13],
    'sigma':[11,16],
    'beta1':[13,19],
    'beta2':[19,30]
    }

### computation

sigma index

In [11]:
""" #TBD -> make this automatic for a function in which you input a dataframe
takeTime=True

for window in windowDurations:
    windowPath=featurespath+'/'+str(samplerate)+'fs/'+str(window)
    for ind, row in signalsMetadata.iterrows():
        subject=row.subjectId
        subjectPath=windowPath+'/'+subject
        signal=signals[subject]
        #need to define the time vector for each signal:
        timepoints=np.arange(len(signal))/samplerate
        if takeTime:
            print("********************************")
            print("reference signal duration: "+str(row.duration))
            print("~"+str(round(row.duration/3600,2))+" hours")
            print("********************************")
            referenceTime=time.time()
        #12. sigma index
        characteristic='sigmaIndex'
        bandName='broadband'
        aux=sigmaindex(signal,window,samplerate)
        saveFeature(aux,window,subject,characteristic,bandName,samplerate,featurespath)
        if takeTime:
            print(characteristic+" "+bandName+": "+str(time.time()-referenceTime))
            referenceTime=time.time()
        takeTime=False
 """

' #TBD -> make this automatic for a function in which you input a dataframe\ntakeTime=True\n\nfor window in windowDurations:\n    windowPath=featurespath+\'/\'+str(samplerate)+\'fs/\'+str(window)\n    for ind, row in signalsMetadata.iterrows():\n        subject=row.subjectId\n        subjectPath=windowPath+\'/\'+subject\n        signal=signals[subject]\n        #need to define the time vector for each signal:\n        timepoints=np.arange(len(signal))/samplerate\n        if takeTime:\n            print("********************************")\n            print("reference signal duration: "+str(row.duration))\n            print("~"+str(round(row.duration/3600,2))+" hours")\n            print("********************************")\n            referenceTime=time.time()\n        #12. sigma index\n        characteristic=\'sigmaIndex\'\n        bandName=\'broadband\'\n        aux=sigmaindex(signal,window,samplerate)\n        saveFeature(aux,window,subject,characteristic,bandName,samplerate,feature

In [12]:
for window in windowDurations:
    windowPath=featurespath+'/'+str(samplerate)+'fs/'+str(window)
    for ind, row in signalsMetadata.iterrows():
        subject=row.subjectId
        subjectPath=windowPath+'/'+subject
        signal=signals[subject]
        #need to define the time vector for each signal:
        timepoints=np.arange(len(signal))/samplerate
        """ #1. Envelopes
        characteristic='envelope'
        for bandName, band in bands.items():
            aux=envelopeHilbert(signal,band,samplerate)
            saveFeature(aux,window,subject,characteristic,bandName,samplerate,featurespath)
        #2. Skewness
        characteristic='skewness'
        bandName='broadband'
        aux=bandSkew(signal,window,'broadband',samplerate)
        saveFeature(aux,window,subject,characteristic,bandName,samplerate,featurespath)
        for bandName, band in bands.items():
            aux=bandSkew(signal,window,band,samplerate)
            saveFeature(aux,window,subject,characteristic,bandName,samplerate,featurespath)
        #3. Kurtosis
        characteristic='kurtosis'
        bandName='broadband'
        aux=bandKurt(signal,window,'broadband',samplerate)
        saveFeature(aux,window,subject,characteristic,bandName,samplerate,featurespath)
        for bandName, band in bands.items():
            aux=bandKurt(signal,window,band,samplerate)
            saveFeature(aux,window,subject,characteristic,bandName,samplerate,featurespath)
        #4. Std
        characteristic='stdev'
        bandName='broadband'
        aux=bandStd(signal,window,'broadband',samplerate)
        saveFeature(aux,window,subject,characteristic,bandName,samplerate,featurespath)
        for bandName, band in bands.items():
            aux=bandStd(signal,window,band,samplerate)
            saveFeature(aux,window,subject,characteristic,bandName,samplerate,featurespath)
        #5. BandRatioRMS
        characteristic='ratioRMS'
        for bandName, band in bands.items():
            aux=bandRatioRMS(signal,band,window,samplerate)
            saveFeature(aux,window,subject,characteristic,bandName,samplerate,featurespath) """
        #6. Hjort Activity
        characteristic='hjortActivity'
        bandName='broadband'
        aux=hjortActivity(signal,window,samplerate)
        saveFeature(aux,window,subject,characteristic,bandName,samplerate,featurespath)
        for bandName, band in bands.items():
            filtered=filterBand(signal,band,samplerate)
            aux=hjortActivity(filtered,window,samplerate)
            saveFeature(aux,window,subject,characteristic,bandName,samplerate,featurespath)
        #7. Hjort Mobility
        characteristic='hjortMobility'
        bandName='broadband'
        aux=hjortMobility(signal,timepoints,window,samplerate)
        saveFeature(aux,window,subject,characteristic,bandName,samplerate,featurespath)
        for bandName, band in bands.items():
            filtered=filterBand(signal,band,samplerate)
            aux=hjortMobility(filtered,timepoints,window,samplerate)
            saveFeature(aux,window,subject,characteristic,bandName,samplerate,featurespath)
        #8. Hjort Complexity
        characteristic='hjortComplexity'
        bandName='broadband'
        aux=hjortComplexity(signal,timepoints,window,samplerate)
        saveFeature(aux,window,subject,characteristic,bandName,samplerate,featurespath)
        for bandName, band in bands.items():
            filtered=filterBand(signal,band,samplerate)
            aux=hjortComplexity(filtered,timepoints,window,samplerate)
            saveFeature(aux,window,subject,characteristic,bandName,samplerate,featurespath)
        #9. PetrosianFractalDimension
        characteristic='petrosian'
        bandName='broadband'
        aux=petrosianFractalDimension(signal,timepoints,window,samplerate)
        saveFeature(aux,window,subject,characteristic,bandName,samplerate,featurespath)
        for bandName, band in bands.items():
            filtered=filterBand(signal,band,samplerate)
            aux=petrosianFractalDimension(filtered,timepoints,window,samplerate)
            saveFeature(aux,window,subject,characteristic,bandName,samplerate,featurespath)
        """ #10. EOD Symmetry (symma)
        characteristic='symmetry'
        bandName='broadband'
        aux=eodSymmetry(signal,window,samplerate)[2]
        saveFeature(aux,window,subject,characteristic,bandName,samplerate,featurespath)
        for bandName, band in bands.items():
            filtered=filterBand(signal,band,samplerate)
            aux=eodSymmetry(filtered,window,samplerate)[2]
            saveFeature(aux,window,subject,characteristic,bandName,samplerate,featurespath)
        #11. LempelZiv
        characteristic='lempelZiv'
        bandName='broadband'
        aux=lempelZivComplexity(signal,window,samplerate)
        saveFeature(aux,window,subject,characteristic,bandName,samplerate,featurespath)
        for bandName, band in bands.items():
            filtered=filterBand(signal,band,samplerate)
            aux=lempelZivComplexity(filtered,window,samplerate)
            saveFeature(aux,window,subject,characteristic,bandName,samplerate,featurespath) """
        #12. sigma index
        characteristic='sigmaIndex'
        bandName='broadband'
        aux=sigmaindex(signal,window,samplerate)
        saveFeature(aux,window,subject,characteristic,bandName,samplerate,featurespath)
        """ #13. sample entropy
        characteristic='sampleEntropy'
        bandName='broadband'
        aux=sampleEntropy(signal,window,samplerate)
        saveFeature(aux,window,subject,characteristic,bandName,samplerate,featurespath)
        for bandName, band in bands.items():
            filtered=filterBand(signal,band,samplerate)
            aux=sampleEntropy(filtered,window,samplerate)
            saveFeature(aux,window,subject,characteristic,bandName,samplerate,featurespath)
        #14. spectral entropy
        characteristic='spectralEntropy'
        bandName='broadband'
        aux=spectralEntropy(signal,[0,50],window,samplerate)
        saveFeature(aux,window,subject,characteristic,bandName,samplerate,featurespath)
        for bandName, band in bands.items():
            aux=spectralEntropy(signal,band,window,samplerate)
            saveFeature(aux,window,subject,characteristic,bandName,samplerate,featurespath)
        #15. median frequency
        characteristic='medianFrequency'
        bandName='broadband'
        aux=medianFrequency(signal,[0,50],window,samplerate)
        saveFeature(aux,window,subject,characteristic,bandName,samplerate,featurespath)
        for bandName, band in bands.items():
            aux=medianFrequency(signal,band,window,samplerate)
            saveFeature(aux,window,subject,characteristic,bandName,samplerate,featurespath)
        #16. relativePower
        characteristic='relativePower'
        for bandName, band in bands.items():
            aux=bandRelativePower(signal,band,window,samplerate)
            saveFeature(aux,window,subject,characteristic,bandName,samplerate,featurespath)
        #17. CTM (default r=0.1)
        characteristic='CTM'
        bandName='broadband'
        aux=centralTendencyMeasure(signal,window,samplerate)
        saveFeature(aux,window,subject,characteristic,bandName,samplerate,featurespath)
        for bandName, band in bands.items():
            filtered=filterBand(signal,band,samplerate)
            aux=centralTendencyMeasure(signal,window,samplerate)
            saveFeature(aux,window,subject,characteristic,bandName,samplerate,featurespath) """
            

C:\Users\Gurdiel\Code\SpinCo\src\spinco.py:329: RuntimeWarning: divide by zero encountered in divide
  mobility=np.sqrt(moveVarCentered(vectorGradient,windowSampleCount)/moveVarCentered(vector,windowSampleCount))
C:\Users\Gurdiel\Code\SpinCo\src\spinco.py:329: RuntimeWarning: invalid value encountered in divide
  mobility=np.sqrt(moveVarCentered(vectorGradient,windowSampleCount)/moveVarCentered(vector,windowSampleCount))
